In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 33.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840629 sha256=69b229844c597fab3aa5aa2d5c5a98cf961881811e0f7bb7e8586a16e504bbef
  Stored in directory: /home/lucaires/.cache/pip/wheels/07/a0/a3/d24c94bf043ab5c7e38c30491199a2a11fef8d2584e6df7fb7
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

24/11/04 16:26:55 WARN Utils: Your hostname, lucaires-IdeaPad-3-15ALC6 resolves to a loopback address: 127.0.1.1; using 192.168.1.12 instead (on interface wlp1s0)
24/11/04 16:26:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/04 16:26:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/11/04 16:27:08 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [7]:
df = spark.read.json("../../datalake/twitter_datascience")

In [8]:
df.show()

+--------------------+--------------------+------------------+------------+
|                data|            includes|              meta|extract_date|
+--------------------+--------------------+------------------+------------+
|[{9, 39, 2024-11-...|{[{2024-11-02T09:...|{1234567890abcdef}|  2024-11-02|
|[{31, 4, 2024-11-...|{[{2024-11-02T14:...|              NULL|  2024-11-02|
|[{48, 83, 2024-11...|{[{2024-11-03T13:...|              NULL|  2024-11-03|
+--------------------+--------------------+------------------+------------+



In [9]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [11]:
from pyspark.sql import functions as f

In [12]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [13]:
df.select(f.explode("data")).show()

+--------------------+
|                 col|
+--------------------+
|{9, 39, 2024-11-0...|
|{18, 20, 2024-11-...|
|{46, 44, 2024-11-...|
|{82, 94, 2024-11-...|
|{71, 81, 2024-11-...|
|{51, 91, 2024-11-...|
|{9, 21, 2024-11-0...|
|{88, 100, 2024-11...|
|{51, 49, 2024-11-...|
|{60, 37, 2024-11-...|
|{31, 4, 2024-11-0...|
|{70, 61, 2024-11-...|
|{20, 51, 2024-11-...|
|{16, 23, 2024-11-...|
|{10, 81, 2024-11-...|
|{84, 100, 2024-11...|
|{15, 49, 2024-11-...|
|{3, 96, 2024-11-0...|
|{80, 60, 2024-11-...|
|{60, 65, 2024-11-...|
+--------------------+
only showing top 20 rows



In [14]:
df.select(f.explode("data").alias("tweets"))\
        .select("tweets.author_id", "tweets.conversation_id",
                "tweets.created_at", "tweets.id",
                "tweets.public_metrics.*", "tweets.text").printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [15]:
tweet_df = df.select(f.explode("data").alias("tweets"))\
        .select("tweets.author_id", "tweets.conversation_id",
                "tweets.created_at", "tweets.id",
                "tweets.public_metrics.*", "tweets.text")

In [16]:
tweet_df.show()

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|        9|             39|2024-11-02T08:20:...| 44|        81|          4|         94|           17|Tweet fictício ge...|
|       18|             20|2024-11-02T22:28:...| 20|        38|         57|          8|           64|Tweet fictício ge...|
|       46|             44|2024-11-02T11:17:...| 91|        90|         45|         17|           91|Um terceiro tweet...|
|       82|             94|2024-11-02T15:27:...| 10|        95|         50|          8|           69|Um terceiro tweet...|
|       71|             81|2024-11-02T21:56:...| 56|        23|         86|         80|           38|Este é um tweet f...|
|       51|     

In [17]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [20]:
df.select(f.explode("includes.users").alias("users")).select("users.*").printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [21]:
user_df = df.select(f.explode("includes.users").alias("users")).select("users.*") 

In [22]:
user_df.show(5)

+--------------------+---+------+--------+
|          created_at| id|  name|username|
+--------------------+---+------+--------+
|2024-11-02T09:42:...|  8|User 1|   user1|
|2024-11-02T04:06:...| 26|User 2|   user2|
|2024-11-02T06:05:...| 39|User 3|   user3|
|2024-11-02T04:13:...| 62|User 4|   user4|
|2024-11-02T19:16:...|  2|User 5|   user5|
+--------------------+---+------+--------+
only showing top 5 rows



In [23]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')
user_df.coalesce(1).write.mode("overwrite").json('output/user')